# Introduction/Business Problem

I would like to compare and determine the best neighborhood in San Antonio, TX for tourists. San Antonio is a big city with a number of tourist sites. It has a local NBA team San Antonio Spurs, which draws NBA fans from all over the world each year. Moreover, with the historical connection with Mexican culture, the Mexican food there is a must try. I hope to give tourists a good recommendation on where to stay given their specific interests in the city.

Whether the focus is on food, tourist sites, or watching NBA games, I would like to let the users to have the power to assign their priority and recommend neighborhoods accordingly. I believe this added flexibility will let this project help many people who visit San Antonio.





# Data

To do so, we need to first define the neighborhoods in San Antonio. The definition of it will come from www.sanantonio.com/neighborhoods, which gives 9 of it. I will first determine the coordinates of these 9 neighborhoods. Then, for each neighborhood, I shall explore the nearby venues. Given that people usually drive in San Antonio, we may be able to set a larger radius, say 2 miles for now. I am also contemplating searching through the city and give scores based on the distance.

Given user-assigned priority, I would like to return the neighborhood with the highest score.

## Data collection

We will start with getting the coordinates of the neighborhoods.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


#!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


#! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'AJCE4HY4IAPY3NGK0GR525M1OA2BF45G0TI4HUONPJVGL0PM' # your Foursquare ID
CLIENT_SECRET = 'J022B41DX1EFZS3ANDVM2IAQAGQQAEZEQU2SZ3WJTHAYWBFB' # your Foursquare Secret
VERSION = '20201031'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AJCE4HY4IAPY3NGK0GR525M1OA2BF45G0TI4HUONPJVGL0PM
CLIENT_SECRET:J022B41DX1EFZS3ANDVM2IAQAGQQAEZEQU2SZ3WJTHAYWBFB


In [76]:
address = 'San Antonio, TX'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude_SA = location.latitude
longitude_SA = location.longitude
print(latitude_SA, longitude_SA)

29.4246002 -98.4951405


In [94]:
neighborhoods = ['Northwest Side', 'Uptown', 'Southside', 'Midtown', 'North Central', 'Alamo Heights', 'East Side', 'Near East Side', 'Downtown']

latitudes = []
longitudes = []

for neighborhood in neighborhoods:
    address_now = neighborhood + ', ' + address
    #geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address_now)
    try:
        latitudes.append(location.latitude)
    except:
        latitudes.append(0)
    try:
        longitudes.append(location.longitude)
    except:
        longitudes.append(0)
    print(latitudes[-1],longitudes[-1])

    
#Since the geolocator cannot find the coordinate of North Central, I have  to key in manually.
latitudes[0] = 29.557154
longitudes[0] = -98.614671

latitudes[1], longitudes[1] = 29.507220, -98.462190
latitudes[2], longitudes[2] = 29.333650, -98.524594
latitudes[4] = 29.497288
longitudes[4] = -98.525972

latitudes[6], longitudes[6] = 29.406447, -98.418599
latitudes[7], longitudes[7] = 29.441394, -98.450757
    
print(latitudes)
print(longitudes)

32.7961145 -97.3380154
26.18995915 -97.6905124964365
26.1564641 -98.1233416
29.4534589 -98.4946316
0 0
29.4832264 -98.468455
33.21574364999999 -97.13152896976189
33.21574364999999 -97.13152896976189
29.4257188 -98.4892023
[29.557154, 29.50722, 29.33365, 29.4534589, 29.497288, 29.4832264, 29.406447, 29.441394, 29.4257188]
[-98.614671, -98.46219, -98.524594, -98.4946316, -98.525972, -98.468455, -98.418599, -98.450757, -98.4892023]


In [95]:
df_hood = pd.DataFrame(data = {'Neighborhood':neighborhoods, 'Latitude': latitudes, 'Longitude':longitudes})
df_hood

,Neighborhood,Latitude,Longitude
0,Northwest Side,29.557154,-98.614671
1,Uptown,29.507220,-98.462190
2,Southside,29.333650,-98.524594
3,Midtown,29.453459,-98.494632
4,North Central,29.497288,-98.525972
5,Alamo Heights,29.483226,-98.468455
6,East Side,29.406447,-98.418599
7,Near East Side,29.441394,-98.450757
8,Downtown,29.425719,-98.489202


We will start by looking for restaurants nearby.

In [96]:
search_query = 'Restaurant'
radius = 3000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=AJCE4HY4IAPY3NGK0GR525M1OA2BF45G0TI4HUONPJVGL0PM&client_secret=J022B41DX1EFZS3ANDVM2IAQAGQQAEZEQU2SZ3WJTHAYWBFB&ll=29.4246002,-98.4951405&v=20201031&query=Restaurant&radius=3000&limit=100'

In [83]:
def getNearbyVenues(names, latitudes, longitudes, search_query, LIMIT, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        
        for v in results:
            try:
                categories_list = v['categories']
            except:
                categories_list = v['venue.categories']

            if len(categories_list) == 0:
                cat_name = None
            else:
                cat_name = categories_list[0]['name']
            
            venues_list.append((name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            cat_name))
        
        #venues_list.append([(
        #    name, 
        #    lat, 
        #    lng, 
        #    v['name'], 
        #    v['location']['lat'], 
        #    v['location']['lng'],  
        #    v['categories'][0]['name']) for v in results])

    #nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues = pd.DataFrame(item for item in venues_list)
    nearby_venues.columns = ['Neighborhood', 
                  'Neighrborhood Latitude', 
                  'Neighrborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [97]:
search_query = 'Restaurant'
df_food = getNearbyVenues(df_hood.Neighborhood, df_hood.Latitude, df_hood.Longitude, search_query, LIMIT, radius)
df_food.head()

Northwest Side
Uptown
Southside
Midtown
North Central
Alamo Heights
East Side
Near East Side
Downtown


,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Rico's Mexican Restaurant,29.555711,-98.614597,Mexican Restaurant
1,Northwest Side,29.557154,-98.614671,Los Jalapenos Mexican Restaurant,29.543234,-98.613792,Mexican Restaurant
2,Northwest Side,29.557154,-98.614671,Don Jose Mexican Restaurant,29.554323,-98.615431,Mexican Restaurant
3,Northwest Side,29.557154,-98.614671,Twin Peaks Restaurant,29.562585,-98.591814,American Restaurant
4,Northwest Side,29.557154,-98.614671,Wynne's Chinese Restaurant,29.543297,-98.613480,Chinese Restaurant


In [98]:
df_food.head(50)

,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Rico's Mexican Restaurant,29.555711,-98.614597,Mexican Restaurant
1,Northwest Side,29.557154,-98.614671,Los Jalapenos Mexican Restaurant,29.543234,-98.613792,Mexican Restaurant
2,Northwest Side,29.557154,-98.614671,Don Jose Mexican Restaurant,29.554323,-98.615431,Mexican Restaurant
3,Northwest Side,29.557154,-98.614671,Twin Peaks Restaurant,29.562585,-98.591814,American Restaurant
4,Northwest Side,29.557154,-98.614671,Wynne's Chinese Restaurant,29.543297,-98.613480,Chinese Restaurant
5,Northwest Side,29.557154,-98.614671,Emmy's Mexican Resturant,29.555339,-98.615297,None
6,Northwest Side,29.557154,-98.614671,Tommy’s Restaurant,29.565912,-98.593985,Mexican Restaurant
7,Northwest Side,29.557154,-98.614671,Kem's Restaurant And Bar,29.568190,-98.596107,American Restaurant
8,Northwest Side,29.557154,-98.614671,A&W Restaurant,29.564376,-98.592562,Fast Food Restaurant
9,Northwest Side,29.557154,-98.614671,A Dong Chinese/Vietnamese Restaurant,29.564133,-98.586757,Vietnamese Restaurant


In [30]:
def getNearbyVenues_explore(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
df_explore = getNearbyVenues_explore(df_hood.Neighborhood, df_hood.Latitude, df_hood.Longitude, radius)

Northwest Side
Uptown
Southside
Midtown
North Central
Alamo Heights
East Side
Near East Side
Downtown


In [34]:
df_explore.head(30)

,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,32.796115,-97.338015,Fort Worth Stockyards National Historic District,32.788794,-97.346710,Neighborhood
1,Northwest Side,32.796115,-97.338015,Fort Worth Stockyards Visitor Center,32.788781,-97.346123,Tourist Information Center
2,Northwest Side,32.796115,-97.338015,Whataburger,32.795620,-97.348297,Burger Joint
3,Northwest Side,32.796115,-97.338015,Hyatt Place Fort Worth Historic Stockyards,32.788260,-97.346345,Hotel
4,Northwest Side,32.796115,-97.338015,Stockyards Station,32.787651,-97.345517,Shopping Plaza
5,Northwest Side,32.796115,-97.338015,Billy Bob's Texas,32.791252,-97.347760,Concert Hall
6,Northwest Side,32.796115,-97.338015,Lonesome Dove Western Bistro,32.788227,-97.349068,New American Restaurant
7,Northwest Side,32.796115,-97.338015,Stockyard Cattle Drive,32.788843,-97.346408,Historic Site
8,Northwest Side,32.796115,-97.338015,The Spice & Tea Exchange of Ft. Worth,32.788234,-97.345670,Herbs & Spices Store
9,Northwest Side,32.796115,-97.338015,Cowtown Coliseum,32.789034,-97.347540,Stadium


In [35]:
df_explore['Venue Category'].unique()

sights = ['Historic Site','History Museum','Trail']

In [101]:
search_query = 'museum'
df_museum = getNearbyVenues(df_hood.Neighborhood, df_hood.Latitude, df_hood.Longitude, search_query, LIMIT, radius=8000)
df_museum.head()

Northwest Side
Uptown
Southside
Midtown
North Central
Alamo Heights
East Side
Near East Side
Downtown


,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Holocaust Memorial Museum San Antonio,29.556899,-98.533308,History Museum
1,Uptown,29.507220,-98.462190,McNay Art Museum,29.485617,-98.456265,Art Museum
2,Uptown,29.507220,-98.462190,McNay Art Museum Stieren Center,29.485668,-98.456248,Art Gallery
3,Uptown,29.507220,-98.462190,San Antonio Museum of Art,29.437324,-98.482115,Museum
4,Uptown,29.507220,-98.462190,Buckhorn Saloon and Museum,29.426340,-98.488814,Monument / Landmark


In [102]:
df_museum
df_museum['Venue Category'].unique()
museum_cat = ['Museum','History Museum','Art Museum','Art Gallery','Cultural Center','Park','Monument / Landmark','Trail','Historic Site','Military Base']

df_museum = df_museum.loc[df_museum['Venue Category'].isin(museum_cat)]
df_museum.head(50)

,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Holocaust Memorial Museum San Antonio,29.556899,-98.533308,History Museum
1,Uptown,29.507220,-98.462190,McNay Art Museum,29.485617,-98.456265,Art Museum
2,Uptown,29.507220,-98.462190,McNay Art Museum Stieren Center,29.485668,-98.456248,Art Gallery
3,Uptown,29.507220,-98.462190,San Antonio Museum of Art,29.437324,-98.482115,Museum
4,Uptown,29.507220,-98.462190,Buckhorn Saloon and Museum,29.426340,-98.488814,Monument / Landmark
5,Uptown,29.507220,-98.462190,US Army Medical Museum,29.461390,-98.466774,History Museum
6,Uptown,29.507220,-98.462190,River Walk Museum Reach Section,29.443455,-98.481302,Park
7,Uptown,29.507220,-98.462190,AMEDD Museum,29.466379,-98.449516,History Museum
8,Uptown,29.507220,-98.462190,San Antonio Fire Museum,29.426185,-98.484118,History Museum
9,Uptown,29.507220,-98.462190,McNay Museum Sculpture Garden,29.485425,-98.456307,Art Museum


In [103]:
search_query = 'sport'
df_sport = getNearbyVenues(df_hood.Neighborhood, df_hood.Latitude, df_hood.Longitude, search_query, LIMIT, radius=3000)
df_sport.head()

Northwest Side
Uptown
Southside
Midtown
North Central
Alamo Heights
East Side
Near East Side
Downtown


,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Cantina Sports Bar & Grill,29.556337,-98.614462,Bar
1,Northwest Side,29.557154,-98.614671,Paris Sports Bar,29.556310,-98.614726,Sports Bar
2,Northwest Side,29.557154,-98.614671,Shenanygans Sports Bar & Lounge,29.546053,-98.615396,Sports Bar
3,Northwest Side,29.557154,-98.614671,Sportsman's Warehouse,29.580245,-98.622464,Sporting Goods Shop
4,Northwest Side,29.557154,-98.614671,Good Sports,29.561869,-98.592948,Sporting Goods Shop


In [104]:
df_sport

,Neighborhood,Neighrborhood Latitude,Neighrborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Northwest Side,29.557154,-98.614671,Cantina Sports Bar & Grill,29.556337,-98.614462,Bar
1,Northwest Side,29.557154,-98.614671,Paris Sports Bar,29.556310,-98.614726,Sports Bar
2,Northwest Side,29.557154,-98.614671,Shenanygans Sports Bar & Lounge,29.546053,-98.615396,Sports Bar
3,Northwest Side,29.557154,-98.614671,Sportsman's Warehouse,29.580245,-98.622464,Sporting Goods Shop
4,Northwest Side,29.557154,-98.614671,Good Sports,29.561869,-98.592948,Sporting Goods Shop
...,...,...,...,...,...,...,...
134,Downtown,29.425719,-98.489202,Reina's Sports Bar,29.419310,-98.526302,Bar
135,Downtown,29.425719,-98.489202,C & L Sports Bar,29.406494,-98.517021,Bar
136,Downtown,29.425719,-98.489202,mission concepcion Sportsplex,29.390810,-98.491693,Basketball Court
137,Downtown,29.425719,-98.489202,Otra Vez Sports Bar,29.390326,-98.503672,Bar


In [125]:
#AT&T center coordinates
coord_att =[29.426809, -98.437391]

In [128]:
venues_map = folium.Map(location=[latitude_SA, longitude_SA], zoom_start=10)

for lat, lng, label in zip(df_food['Venue Latitude'], df_food['Venue Longitude'], df_food['Venue']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
for lat, lng, label in zip(df_museum['Venue Latitude'], df_museum['Venue Longitude'], df_museum['Venue']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)

for lat, lng, label in zip(df_hood['Latitude'], df_hood['Longitude'], df_hood['Neighborhood']):
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
    
folium.CircleMarker(
    [coord_att[0],coord_att[1]],
    radius = 8,
    popup = "AT&T Center",
    fill = True,
    color = "cyan",
    fill_color = 'cyan',
    fill_opacity = 0.6).add_to(venues_map)
    

In [129]:
venues_map